# Keras 预处理层与自定义数据处理

Keras 预处理层将数据转换逻辑封装为模型的一部分，确保训练和推理时的一致性。本教程深入讲解内置预处理层和自定义层的实现。

## 核心知识点

1. 自定义预处理层的设计与实现
2. Keras 内置 Normalization、Rescaling 层
3. 分类特征编码：StringLookup、CategoryEncoding、Embedding
4. 预处理层与模型的集成策略

In [ ]:
import tensorflow as tf
import numpy as np
import keras
from keras import layers

# 固定随机种子
SEED = 42
tf.random.set_seed(SEED)
np.random.seed(SEED)

print(f"TensorFlow: {tf.__version__}")
print(f"Keras: {keras.__version__}")

## 1. 自定义预处理层

### 1.1 为什么需要自定义层

内置层覆盖常见场景，但特殊需求需要自定义实现：

- 特定领域的归一化方法
- 复杂的特征工程逻辑
- 需要从数据中学习参数的转换

### 1.2 自定义标准化层

Z-score 标准化公式：

$$z = \frac{x - \mu}{\sigma}$$

其中 $\mu$ 是均值，$\sigma$ 是标准差。

In [ ]:
class CustomStandardization(layers.Layer):
    """自定义 Z-score 标准化层
    
    将输入数据转换为均值为 0、标准差为 1 的分布。
    支持通过 adapt() 方法从数据中学习统计量。
    
    Attributes:
        epsilon: 防止除零的小常数
        means_: 学习到的特征均值
        stds_: 学习到的特征标准差
    """
    
    def __init__(self, epsilon=1e-7, **kwargs):
        super().__init__(**kwargs)
        self.epsilon = epsilon
        self.means_ = None
        self.stds_ = None
    
    def adapt(self, data):
        """从数据中学习均值和标准差
        
        Args:
            data: 训练数据，numpy 数组或 tf.Tensor
        """
        data = np.asarray(data, dtype=np.float32)
        self.means_ = np.mean(data, axis=0, keepdims=True)
        self.stds_ = np.std(data, axis=0, keepdims=True)
        # 处理常数特征（标准差为 0）
        self.stds_ = np.where(self.stds_ < self.epsilon, 1.0, self.stds_)
    
    def call(self, inputs):
        """应用标准化转换"""
        if self.means_ is None:
            raise RuntimeError("层未初始化，请先调用 adapt()")
        inputs = tf.cast(inputs, tf.float32)
        return (inputs - self.means_) / self.stds_
    
    def inverse_transform(self, normalized):
        """逆变换：将标准化数据还原到原始尺度"""
        return normalized * self.stds_ + self.means_
    
    def get_config(self):
        """序列化配置"""
        config = super().get_config()
        config.update({"epsilon": self.epsilon})
        return config

In [ ]:
# 测试自定义标准化层
# 构造不同尺度的特征
test_data = np.array([
    [100.0, 0.1, 50.0],
    [120.0, 0.2, 60.0],
    [80.0, 0.15, 40.0],
    [110.0, 0.25, 55.0],
    [90.0, 0.12, 45.0],
], dtype=np.float32)

print("原始数据:")
print(test_data)
print(f"\n各特征均值: {test_data.mean(axis=0)}")
print(f"各特征标准差: {test_data.std(axis=0)}")

# 创建并适配层
standardizer = CustomStandardization()
standardizer.adapt(test_data)

# 应用标准化
normalized = standardizer(test_data)
print(f"\n标准化后均值: {normalized.numpy().mean(axis=0)}")
print(f"标准化后标准差: {normalized.numpy().std(axis=0)}")

# 验证逆变换
restored = standardizer.inverse_transform(normalized)
print(f"\n逆变换误差: {np.abs(restored.numpy() - test_data).max():.2e}")

### 1.3 自定义归一化层

Min-Max 归一化公式：

$$x_{norm} = \frac{x - x_{min}}{x_{max} - x_{min}}$$

将数据缩放到 [0, 1] 或指定范围。

In [ ]:
class CustomMinMaxNormalization(layers.Layer):
    """自定义 Min-Max 归一化层
    
    将数据缩放到指定范围，默认 [0, 1]。
    """
    
    def __init__(self, feature_range=(0, 1), epsilon=1e-7, **kwargs):
        super().__init__(**kwargs)
        self.feature_range = feature_range
        self.epsilon = epsilon
        self.data_min_ = None
        self.data_max_ = None
    
    def adapt(self, data):
        """从数据中学习最小值和最大值"""
        data = np.asarray(data, dtype=np.float32)
        self.data_min_ = np.min(data, axis=0, keepdims=True)
        self.data_max_ = np.max(data, axis=0, keepdims=True)
    
    def call(self, inputs):
        """应用归一化转换"""
        if self.data_min_ is None:
            raise RuntimeError("层未初始化，请先调用 adapt()")
        
        inputs = tf.cast(inputs, tf.float32)
        data_range = self.data_max_ - self.data_min_
        # 处理常数特征
        data_range = tf.where(data_range < self.epsilon, 1.0, data_range)
        
        # 缩放到 [0, 1]
        scaled = (inputs - self.data_min_) / data_range
        
        # 转换到目标范围
        min_val, max_val = self.feature_range
        return scaled * (max_val - min_val) + min_val
    
    def get_config(self):
        config = super().get_config()
        config.update({
            "feature_range": self.feature_range,
            "epsilon": self.epsilon,
        })
        return config

In [ ]:
# 测试归一化层
normalizer = CustomMinMaxNormalization()
normalizer.adapt(test_data)

normalized = normalizer(test_data)
print(f"归一化到 [0, 1]:")
print(f"  范围: [{normalized.numpy().min():.4f}, {normalized.numpy().max():.4f}]")

# 测试自定义范围
normalizer_custom = CustomMinMaxNormalization(feature_range=(-1, 1))
normalizer_custom.adapt(test_data)
normalized_custom = normalizer_custom(test_data)
print(f"\n归一化到 [-1, 1]:")
print(f"  范围: [{normalized_custom.numpy().min():.4f}, {normalized_custom.numpy().max():.4f}]")

## 2. Keras 内置预处理层

### 2.1 Normalization 层

Keras 内置的 Z-score 标准化实现，功能完整且经过优化。

In [ ]:
# 生成不同尺度的特征数据
np.random.seed(42)
train_features = np.random.randn(1000, 5).astype(np.float32)
train_features[:, 0] *= 100   # 特征 0: 大尺度
train_features[:, 1] *= 0.01  # 特征 1: 小尺度
train_features[:, 2] += 50    # 特征 2: 偏移

print("原始特征统计:")
for i in range(5):
    print(f"  特征 {i}: 均值={train_features[:, i].mean():8.4f}, "
          f"标准差={train_features[:, i].std():8.4f}")

In [ ]:
# 使用 Keras Normalization 层
keras_normalizer = layers.Normalization(axis=-1)
keras_normalizer.adapt(train_features)

# 查看学习到的参数
print("学习到的均值:")
print(f"  {keras_normalizer.mean.numpy().flatten()}")
print("\n学习到的方差:")
print(f"  {keras_normalizer.variance.numpy().flatten()}")

# 应用标准化
normalized = keras_normalizer(train_features)
print("\n标准化后统计:")
for i in range(5):
    col = normalized[:, i].numpy()
    print(f"  特征 {i}: 均值={col.mean():8.4f}, 标准差={col.std():8.4f}")

### 2.2 Rescaling 层

线性缩放层，常用于图像像素值归一化。

公式：$y = x \times scale + offset$

In [ ]:
# 模拟图像数据 (0-255 像素值)
fake_images = np.random.randint(0, 256, size=(4, 28, 28, 1)).astype(np.float32)
print(f"原始像素范围: [{fake_images.min()}, {fake_images.max()}]")

# 缩放到 [0, 1]
rescaler_01 = layers.Rescaling(scale=1.0/255)
scaled_01 = rescaler_01(fake_images)
print(f"缩放到 [0, 1]: [{scaled_01.numpy().min():.4f}, {scaled_01.numpy().max():.4f}]")

# 缩放到 [-1, 1]（常用于预训练模型）
rescaler_11 = layers.Rescaling(scale=1.0/127.5, offset=-1)
scaled_11 = rescaler_11(fake_images)
print(f"缩放到 [-1, 1]: [{scaled_11.numpy().min():.4f}, {scaled_11.numpy().max():.4f}]")

## 3. 分类特征编码

### 3.1 StringLookup：字符串索引化

将字符串映射为整数索引，处理词汇表外（OOV）的输入。

In [ ]:
# 定义词汇表
weather_vocab = ["晴天", "多云", "阴天", "小雨", "大雨", "雪"]

# 创建查找层
weather_lookup = layers.StringLookup(
    vocabulary=weather_vocab,
    output_mode="int"
)

# 查看词汇表（索引 0 是 OOV）
print("词汇表:")
for i, word in enumerate(weather_lookup.get_vocabulary()):
    print(f"  索引 {i}: {word}")

# 测试转换
test_weather = tf.constant(["晴天", "小雨", "未知天气", "多云"])
indices = weather_lookup(test_weather)

print(f"\n输入: {[w.numpy().decode() for w in test_weather]}")
print(f"索引: {indices.numpy()}")
print("注意: '未知天气' 被映射到索引 0 (OOV)")

### 3.2 CategoryEncoding：类别编码

将整数索引转换为独热编码或多热编码。

**适用场景**：低基数分类特征（类别数 < 50）

In [ ]:
# 创建独热编码层
# num_tokens = 词汇表大小 + 1 (OOV)
category_encoder = layers.CategoryEncoding(
    num_tokens=len(weather_vocab) + 1,
    output_mode="one_hot"
)

# 应用编码
one_hot = category_encoder(indices)

print("独热编码结果:")
for i, (weather, idx) in enumerate(zip(test_weather.numpy(), indices.numpy())):
    print(f"  {weather.decode()}: 索引={idx}, 编码={one_hot[i].numpy()}")

In [ ]:
# 封装完整的字符串到独热编码流程
def create_string_to_onehot(vocabulary):
    """创建字符串到独热编码的转换函数
    
    Args:
        vocabulary: 类别词汇表
        
    Returns:
        编码函数
    """
    lookup = layers.StringLookup(vocabulary=vocabulary, output_mode="int")
    encoder = layers.CategoryEncoding(
        num_tokens=len(vocabulary) + 1,
        output_mode="one_hot"
    )
    
    def encode(inputs):
        return encoder(lookup(inputs))
    
    return encode


# 使用示例
color_encoder = create_string_to_onehot(["红", "绿", "蓝", "黄"])
colors = tf.constant(["红", "蓝", "白"])  # "白" 是未知类别
encoded = color_encoder(colors)

print("颜色编码:")
for color, enc in zip(colors.numpy(), encoded.numpy()):
    print(f"  {color.decode()}: {enc}")

### 3.3 Embedding 层：高基数特征

当类别数量很大时（如用户 ID、商品 ID），独热编码产生高维稀疏向量。Embedding 层将离散索引映射到低维稠密向量。

**优势**：
- 参数高效：N 个类别只需 N × D 个参数（D 是嵌入维度）
- 可学习相似性：相似类别的嵌入向量会在训练中趋近

In [ ]:
# 模拟用户 ID 场景
NUM_USERS = 10000
EMBEDDING_DIM = 16

# 创建嵌入层
user_embedding = layers.Embedding(
    input_dim=NUM_USERS,
    output_dim=EMBEDDING_DIM,
    name="user_embedding"
)

# 查询嵌入向量
user_ids = tf.constant([0, 100, 5000, 9999])
embeddings = user_embedding(user_ids)

print(f"输入用户 ID: {user_ids.numpy()}")
print(f"嵌入向量形状: {embeddings.shape}")
print(f"\n用户 0 的嵌入向量:")
print(f"  {embeddings[0].numpy()[:8]}...")

In [ ]:
# 参数量对比
print("10000 个类别的编码方式对比:")
print(f"  独热编码: 0 参数，但输入维度为 10000")
print(f"  Embedding (dim=16): {NUM_USERS * 16:,} 参数")
print(f"  Embedding (dim=32): {NUM_USERS * 32:,} 参数")
print(f"\n嵌入层参数量 = 类别数 × 嵌入维度")

## 4. 预处理层与模型集成

### 4.1 集成的优势

将预处理层放入模型：

1. **一致性**：训练和推理使用相同的预处理逻辑
2. **可移植性**：`model.save()` 会保存预处理参数
3. **性能**：预处理可在 GPU 上执行

In [ ]:
# 生成模拟数据
np.random.seed(42)
X = np.random.randn(500, 4).astype(np.float32)
X[:, 0] *= 100  # 不同尺度
X[:, 1] += 50
y = (X[:, 0] * 0.5 + X[:, 1] * 0.3 + np.random.randn(500) * 10).astype(np.float32)

print(f"特征形状: {X.shape}")
print(f"标签形状: {y.shape}")

In [ ]:
# 创建并适配归一化层
normalizer = layers.Normalization(axis=-1)
normalizer.adapt(X)

# 构建带预处理层的模型
model = keras.Sequential([
    layers.Input(shape=(4,), name="input"),
    normalizer,  # 预处理层
    layers.Dense(32, activation="relu"),
    layers.Dense(16, activation="relu"),
    layers.Dense(1),
], name="model_with_preprocessing")

model.summary()

In [ ]:
# 编译并训练（使用简化参数快速验证）
model.compile(
    optimizer="adam",
    loss="mse",
    metrics=["mae"]
)

history = model.fit(
    X, y,
    epochs=10,
    batch_size=32,
    validation_split=0.2,
    verbose=1
)

print(f"\n最终训练损失: {history.history['loss'][-1]:.4f}")
print(f"最终验证损失: {history.history['val_loss'][-1]:.4f}")

In [ ]:
# 推理时无需手动预处理
test_input = np.array([[150.0, 60.0, 0.5, -0.5]], dtype=np.float32)
print(f"测试输入（原始尺度）: {test_input}")

# 模型内部自动应用归一化
prediction = model.predict(test_input, verbose=0)
print(f"预测结果: {prediction[0, 0]:.4f}")

### 4.2 混合特征处理

实际场景常需同时处理数值特征和分类特征。

In [ ]:
def build_mixed_feature_model(numeric_dim, category_vocab_sizes, embedding_dims):
    """构建混合特征处理模型
    
    Args:
        numeric_dim: 数值特征维度
        category_vocab_sizes: 各分类特征的词汇表大小
        embedding_dims: 各分类特征的嵌入维度
        
    Returns:
        Keras 模型
    """
    # 数值输入
    numeric_input = layers.Input(shape=(numeric_dim,), name="numeric")
    normalized_numeric = layers.Normalization(axis=-1)(numeric_input)
    
    # 分类输入和嵌入
    category_inputs = []
    category_embeddings = []
    
    for i, (vocab_size, emb_dim) in enumerate(zip(category_vocab_sizes, embedding_dims)):
        cat_input = layers.Input(shape=(1,), dtype="int32", name=f"category_{i}")
        category_inputs.append(cat_input)
        
        embedding = layers.Embedding(vocab_size, emb_dim)(cat_input)
        embedding = layers.Flatten()(embedding)
        category_embeddings.append(embedding)
    
    # 拼接所有特征
    all_features = layers.Concatenate()([normalized_numeric] + category_embeddings)
    
    # 全连接层
    x = layers.Dense(64, activation="relu")(all_features)
    x = layers.Dense(32, activation="relu")(x)
    output = layers.Dense(1)(x)
    
    return keras.Model(
        inputs=[numeric_input] + category_inputs,
        outputs=output,
        name="mixed_feature_model"
    )


# 创建模型
mixed_model = build_mixed_feature_model(
    numeric_dim=5,
    category_vocab_sizes=[100, 50],
    embedding_dims=[8, 4]
)

mixed_model.summary()

## 总结

### 预处理层对比

| 层类型 | 用途 | 适用场景 |
|--------|------|----------|
| `Normalization` | Z-score 标准化 | 数值特征，不同尺度 |
| `Rescaling` | 线性缩放 | 图像像素值 |
| `StringLookup` | 字符串转索引 | 分类特征预处理 |
| `CategoryEncoding` | 独热/多热编码 | 低基数分类特征 |
| `Embedding` | 嵌入向量 | 高基数分类特征 |

### 最佳实践

1. **预处理集成**: 将预处理层放入模型，确保训练和推理一致
2. **adapt 时机**: 仅使用训练数据调用 `adapt()`，避免数据泄露
3. **特征选择**: 低基数用独热编码，高基数用嵌入
4. **嵌入维度**: 经验法则 `dim = min(50, vocab_size // 2)`

### 参考文档

- [Keras 预处理层指南](https://keras.io/guides/preprocessing_layers/)
- [结构化数据处理](https://www.tensorflow.org/tutorials/structured_data/preprocessing_layers)